In [1]:
!git clone https://github.com/wdrink/M2TR-Multi-modal-Multi-scale-Transformers-for-Deepfake-Detection.git

Cloning into 'M2TR-Multi-modal-Multi-scale-Transformers-for-Deepfake-Detection'...
remote: Enumerating objects: 119, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 119 (delta 52), reused 79 (delta 31), pack-reused 0 (from 0)
Receiving objects: 100% (119/119), 1.80 MiB | 27.85 MiB/s, done.
Resolving deltas: 100% (52/52), done.


In [2]:
%cd /kaggle/working/M2TR-Multi-modal-Multi-scale-Transformers-for-Deepfake-Detection

/kaggle/working/M2TR-Multi-modal-Multi-scale-Transformers-for-Deepfake-Detection


In [3]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
ERROR: Ignored the following versions that require a different python version: 1.6.2 Requires-Python >=3.7,<3.10; 1.6.3 Requires-Python >=3.7,<3.10; 1.7.0 Requires-Python >=3.7,<3.10; 1.7.1 Requires-Python >=3.7,<3.10
ERROR: Could not find a version that satisfies the requirement torch==1.11.0+cu113 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1)
ERROR: No matching distribution found for torch==1.11.0+cu113


In [4]:
!pip install fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61395 sha256=dfec3cedd194b473ac258897e6aa8d013e69dfa2208afbe63c3af8df50fd7fe9
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31529 sha256=5740d421bd007e252075b4a7da83853d0908d9872bb53e83f834361d3e733a8d
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath


In [5]:
!pip install simplejson

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 5.3 MB/s eta 0:00:00


In [6]:
!pip install -U albumentations

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.1/80.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.9/269.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 632.7/632.7 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.5/303.5 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: albucore
    Found existing installation: albucore 0.0.16
    Uninstalling albucore-0.0.16:
      Successfully uninstalled albucore-0.0.16
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.4.15
    Uninstalling albumentations-1.4.15:
      Successfully uninstalled albumentations-1.4.15


In [7]:
import numpy as np
import pandas as pd
import csv
import os
import imageio.v2 as imageio

import fvcore
import simplejson

import torchvision
import torchvision.transforms as transforms

from torch.utils.data import Dataset, DataLoader, random_split

In [8]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
#!python run.py --cfg m2tr.yaml

In [9]:
import math

import torch
import torch.fft
import torch.nn as nn
import torch.nn.functional as F

from M2TR.utils.registries import MODEL_REGISTRY

from M2TR.models.base import BaseNetwork
from M2TR.models.xception import Xception
from M2TR.models.efficientnet import EfficientNet
from M2TR.models.modules.head import Classifier2D, Localizer
from M2TR.models.modules.transformer_block import FeedForward2D



class GlobalFilter(nn.Module):
    def __init__(self, dim=32, h=80, w=41, fp32fft=True):
        super().__init__()
        self.complex_weight = nn.Parameter(
            torch.randn(h, w, dim, 2, dtype=torch.float32) * 0.02
        )
        self.w = w
        self.h = h
        self.fp32fft = fp32fft

    def forward(self, x):
        b, _, a, b = x.size()
        x = x.permute(0, 2, 3, 1).contiguous()

        if self.fp32fft:
            dtype = x.dtype
            x = x.to(torch.float32)

        x = torch.fft.rfft2(x, dim=(1, 2), norm="ortho")
        weight = torch.view_as_complex(self.complex_weight)
        x = x * weight
        x = torch.fft.irfft2(x, s=(a, b), dim=(1, 2), norm="ortho")

        if self.fp32fft:
            x = x.to(dtype)

        x = x.permute(0, 3, 1, 2).contiguous()

        return x


class FreqBlock(nn.Module):
    def __init__(self, dim, h=80, w=41, fp32fft=True):
        super().__init__()
        self.filter = GlobalFilter(dim, h=h, w=w, fp32fft=fp32fft)
        self.feed_forward = FeedForward2D(in_channel=dim, out_channel=dim)

    def forward(self, x):
        x = x + self.feed_forward(self.filter(x))
        return x


def attention(query, key, value):
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(
        query.size(-1)
    )
    p_attn = F.softmax(scores, dim=-1)
    p_val = torch.matmul(p_attn, value)
    return p_val, p_attn


class MultiHeadedAttention(nn.Module):
    """
    Take in model size and number of heads.
    """

    def __init__(self, patchsize, d_model):
        super().__init__()
        self.patchsize = patchsize
        self.query_embedding = nn.Conv2d(
            d_model, d_model, kernel_size=1, padding=0
        )
        self.value_embedding = nn.Conv2d(
            d_model, d_model, kernel_size=1, padding=0
        )
        self.key_embedding = nn.Conv2d(
            d_model, d_model, kernel_size=1, padding=0
        )
        self.output_linear = nn.Sequential(
            nn.Conv2d(d_model, d_model, kernel_size=3, padding=1),
            nn.BatchNorm2d(d_model),
            nn.LeakyReLU(0.2, inplace=True),
        )

    def forward(self, x):
        b, c, h, w = x.size()
        d_k = c // len(self.patchsize)
        output = []
        _query = self.query_embedding(x)
        _key = self.key_embedding(x)
        _value = self.value_embedding(x)
        attentions = []
        for (width, height), query, key, value in zip(
            self.patchsize,
            torch.chunk(_query, len(self.patchsize), dim=1),
            torch.chunk(_key, len(self.patchsize), dim=1),
            torch.chunk(_value, len(self.patchsize), dim=1),
        ):
            out_w, out_h = w // width, h // height

            # 1) embedding and reshape
            query = query.view(b, d_k, out_h, height, out_w, width)
            query = (
                query.permute(0, 2, 4, 1, 3, 5)
                .contiguous()
                .view(b, out_h * out_w, d_k * height * width)
            )
            key = key.view(b, d_k, out_h, height, out_w, width)
            key = (
                key.permute(0, 2, 4, 1, 3, 5)
                .contiguous()
                .view(b, out_h * out_w, d_k * height * width)
            )
            value = value.view(b, d_k, out_h, height, out_w, width)
            value = (
                value.permute(0, 2, 4, 1, 3, 5)
                .contiguous()
                .view(b, out_h * out_w, d_k * height * width)
            )

            y, _ = attention(query, key, value)

            # 3) "Concat" using a view and apply a final linear.
            y = y.view(b, out_h, out_w, d_k, height, width)
            y = y.permute(0, 3, 1, 4, 2, 5).contiguous().view(b, d_k, h, w)
            attentions.append(y)
            output.append(y)

        output = torch.cat(output, 1)
        self_attention = self.output_linear(output)

        return self_attention


class TransformerBlock(nn.Module):
    """
    Transformer = MultiHead_Attention + Feed_Forward with sublayer connection
    """

    def __init__(self, patchsize, in_channel=256):
        super().__init__()
        self.attention = MultiHeadedAttention(patchsize, d_model=in_channel)
        self.feed_forward = FeedForward2D(
            in_channel=in_channel, out_channel=in_channel
        )

    def forward(self, rgb):
        self_attention = self.attention(rgb)
        output = rgb + self_attention
        output = output + self.feed_forward(output)
        return output


class CMA_Block(nn.Module):
    def __init__(self, in_channel, hidden_channel, out_channel):
        super(CMA_Block, self).__init__()

        self.conv1 = nn.Conv2d(
            in_channel, hidden_channel, kernel_size=1, stride=1, padding=0
        )
        self.conv2 = nn.Conv2d(
            in_channel, hidden_channel, kernel_size=1, stride=1, padding=0
        )
        self.conv3 = nn.Conv2d(
            in_channel, hidden_channel, kernel_size=1, stride=1, padding=0
        )

        self.scale = hidden_channel ** -0.5

        self.conv4 = nn.Sequential(
            nn.Conv2d(
                hidden_channel, out_channel, kernel_size=1, stride=1, padding=0
            ),
            nn.BatchNorm2d(out_channel),
            nn.LeakyReLU(0.2, inplace=True),
        )

    def forward(self, rgb, freq):
        _, _, h, w = rgb.size()

        q = self.conv1(rgb)
        k = self.conv2(freq)
        v = self.conv3(freq)

        q = q.view(q.size(0), q.size(1), q.size(2) * q.size(3)).transpose(
            -2, -1
        )
        k = k.view(k.size(0), k.size(1), k.size(2) * k.size(3))

        attn = torch.matmul(q, k) * self.scale
        m = attn.softmax(dim=-1)

        v = v.view(v.size(0), v.size(1), v.size(2) * v.size(3)).transpose(
            -2, -1
        )
        z = torch.matmul(m, v)
        z = z.view(z.size(0), h, w, -1)
        z = z.permute(0, 3, 1, 2).contiguous()

        output = rgb + self.conv4(z)

        return output


class PatchTrans(BaseNetwork):
    def __init__(self, in_channel, in_size):
        super(PatchTrans, self).__init__()
        self.in_size = in_size

        patchsize = [
            (in_size, in_size),
            (in_size // 2, in_size // 2),
            (in_size // 4, in_size // 4),
            (in_size // 8, in_size // 8),
        ]

        self.t = TransformerBlock(patchsize, in_channel=in_channel)

    def forward(self, enc_feat):
        output = self.t(enc_feat)
        return output


#@MODEL_REGISTRY.register()
class M2TR(BaseNetwork):
    def __init__(self, model_cfg):
        super(M2TR, self).__init__()
        img_size = model_cfg["IMG_SIZE"]
        backbone = model_cfg["BACKBONE"]
        texture_layer = model_cfg["TEXTURE_LAYER"]
        feature_layer = model_cfg["FEATURE_LAYER"]
        depth = model_cfg["DEPTH"]
        num_classes = model_cfg["NUM_CLASSES"]
        drop_ratio = model_cfg["DROP_RATIO"]
        has_decoder = model_cfg["HAS_DECODER"]

        freq_h = img_size // 4
        freq_w = freq_h // 2 + 1

        if "xception" in backbone:
            self.model = Xception(num_classes)
        elif backbone.split("-")[0] == "efficientnet":
            self.model = EfficientNet({'NAME': backbone, 'PRETRAINED': True})

        self.texture_layer = texture_layer
        self.feature_layer = feature_layer

        with torch.no_grad():
            input = {"img": torch.zeros(1, 3, img_size, img_size)}
            layers = self.model(input)
        texture_dim = layers[self.texture_layer].shape[1]
        feature_dim = layers[self.feature_layer].shape[1]

        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(
                nn.ModuleList(
                    [
                        PatchTrans(in_channel=texture_dim, in_size=freq_h),
                        FreqBlock(dim=texture_dim, h=freq_h, w=freq_w),
                        CMA_Block(
                            in_channel=texture_dim,
                            hidden_channel=texture_dim,
                            out_channel=texture_dim,
                        ),
                    ]
                )
            )

        self.classifier = Classifier2D(
            feature_dim, num_classes, drop_ratio, "softmax"
        )

        self.has_decoder = has_decoder
        if self.has_decoder:
            self.decoder = Localizer(texture_dim, 1)

    def forward(self, x):
        rgb = x #["img"]
        B = rgb.size(0)

        layers = {}
        rgb = self.model.extract_textures(rgb, layers)

        for attn, filter, cma in self.layers:
            rgb = attn(rgb)
            freq = filter(rgb)
            rgb = cma(rgb, freq)

        features = self.model.extract_features(rgb, layers)
        features = F.adaptive_avg_pool2d(features, (1, 1))
        features = features.view(B, features.size(1))

        logits = self.classifier(features)

        if self.has_decoder:
            mask = self.decoder(rgb)
            mask = mask.squeeze(-1)

        else:
            mask = None

        output = {"logits": logits, "mask": mask, "features:": features}
        return output




In [10]:
from torchsummary import summary


model_cfg = {"IMG_SIZE":256,
             "BACKBONE": 'efficientnet-b4',
             "TEXTURE_LAYER":'b2',
             "FEATURE_LAYER": 'final',
             "DEPTH":4,
             "NUM_CLASSES":2,
             "DROP_RATIO":0.5,
             "HAS_DECODER":False}


model = M2TR(model_cfg)
model.cuda()
# summary(model, input_size=(3, 256, 256), batch_size=12, device="cuda")

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/adv-efficientnet-b4-44fb3a87.pth" to /root/.cache/torch/hub/checkpoints/adv-efficientnet-b4-44fb3a87.pth
100%|██████████| 74.4M/74.4M [00:01<00:00, 51.8MB/s]


Loaded pretrained weights for efficientnet-b4
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [12, 3, 257, 257]               0
Conv2dStaticSamePadding-2         [12, 48, 128, 128]           1,296
       BatchNorm2d-3         [12, 48, 128, 128]              96
MemoryEfficientSwish-4         [12, 48, 128, 128]               0
         ZeroPad2d-5         [12, 48, 130, 130]               0
Conv2dStaticSamePadding-6         [12, 48, 128, 128]             432
       BatchNorm2d-7         [12, 48, 128, 128]              96
MemoryEfficientSwish-8         [12, 48, 128, 128]               0
          Identity-9             [12, 48, 1, 1]               0
Conv2dStaticSamePadding-10             [12, 12, 1, 1]             588
MemoryEfficientSwish-11             [12, 12, 1, 1]               0
         Identity-12             [12, 12, 1, 1]               0
Conv2dStaticSamePadding-13        

In [11]:
def normalize(img, eps=1e-8):
    min = np.min(img)
    max = np.max(img)

    norm_img = (img-min)/(max-min+eps)
    
    return norm_img




def read_data(path):
    img_ids=[]
    targets=[]
    
    with open (path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            if len(row)==2:
                img_id, target = row
                img_ids.append(img_id)
                targets.append(target)
                
    return img_ids, targets



def onehot_encode(logits):
    num_classes = logits.shape[1]
    max_indices = torch.argmax(logits, dim=1)
    
    result = F.one_hot(max_indices, num_classes=num_classes)
    return(result)
        

In [12]:
class CreateDataset(Dataset):
    def __init__(self, dir_path, label_path, transform=None):
        self.data_path = dir_path
        self.label_path = label_path
        self.transform = transform
        
        self.imgfiles, self.labels = read_data(label_path)
        

    def __len__(self):
        return len(self.imgfiles)
        
    def __getitem__(self, index):
        imgpath = os.path.join(self.data_path, self.imgfiles[index])
        img = imageio.imread(imgpath)

        img = normalize(img)
        
        if self.transform is not None:
            img = self.transform(img)

        label = int(self.labels[index])
        label_arr = np.zeros(2)
        label_arr[label] = 1
        
        return img, label_arr

In [13]:
data_folder = '/kaggle/input/deepfake/phase1/trainset'
label_path = '/kaggle/input/deepfake/phase1/trainset_label.txt'

In [14]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.CenterCrop(256),
    transforms.ConvertImageDtype(torch.float32)
])

dataset=CreateDataset(data_folder, label_path, transform)

train_size = int(0.9*len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset,[train_size, val_size])

train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=100, shuffle=False)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_epochs= 3
lr = 0.0001

total_loss = 0
trainloss_list = []

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.75, patience=15, min_lr=1e-5)

criterion = nn.CrossEntropyLoss()

for epoch in range (num_epochs):
    model.train()
    for (idx,(x_train, y_true)) in enumerate(train_dataloader):
        
        
        x_train, y_true = x_train.to(device), y_true.to(device)
    
        optimizer.zero_grad()
        
        output = model(x_train)
        y_pred = output['logits']

        
    
        loss = criterion(y_pred, y_true)
        loss.backward()
    
        optimizer.step()
    
        total_loss += loss

        if idx%1000==0:
            print(f'\trunning epoch {epoch+1} ......')
    
    train_loss = total_loss/len(train_dataset)
    trainloss_list.append(train_loss.item())
    
    
    val_loss = 0
    valloss_list = []

    
    model.eval()
    with torch.no_grad():
        for (x_val, y_true) in val_dataloader:
            
            x_val, y_true = x_val.to(device), y_true.to(device).reshape(-1,1)
            
            output = model(x_val)
            y_pred = output['logits']
    
            val_loss += criterion(y_pred, y_true)
    
            scheduler.step(val_loss)
            '''
            class_pred = sigmoid_threshold(y_pred)
            
            tn, fp, fn, tp = confusion_matrix(y_true, class_pred).ravel()
            TP.extend(tp)
            FP.extend(fp)
            TN.extend(tn)
            FN.extend(fn)
            '''
        val_loss = val_loss/len(val_dataset)
        valloss_list.append(val_loss.item())

        
    
    current_lr = optimizer.param_groups[0]['lr']
    
    print(f'{epoch+1}|{num_epochs} train loss: {train_loss},  val loss: {val_loss},  lr: {current_lr:.6f}')

	running epoch 1 ......
	running epoch 1 ......
	running epoch 1 ......
	running epoch 1 ......
	running epoch 1 ......
	running epoch 1 ......
	running epoch 1 ......
	running epoch 1 ......
	running epoch 1 ......
	running epoch 1 ......
	running epoch 1 ......
	running epoch 1 ......
	running epoch 1 ......
	running epoch 1 ......


output = {"logits": logits, "mask": mask, "features:": features}

In [ ]:
path = '/kaggle/working/model_weights.pth'
torch.save(model.state_dict(), path)

In [ ]:
# print(f'labels: {label}\n')
# print('predictions:', onehot_encode(output['logits']))

In [ ]:
# softmax = nn.Softmax()
# pred = torch.tensor([[-0.1362, -0.0531],
#         [ 0.0268,  0.0570]])

In [ ]:
# torch.argmax(pred, dim=1)